Test with synthetic data
=============

In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
basedir = '../'
sys.path.append(basedir)

import cPickle
import sys
import time

from lda.lda import LDA
from numpy import int32
from numpy.random import RandomState

from lda_cgs_numba import sample_numba
from lda_cgs_numpy import sample_numpy
from lda_generate_data import LdaDataGenerator
import lda_utils as utils
import numpy as np
import pandas as pd
import pylab as plt
from scipy.special import psi
from lda_utils import estimate_alpha_from_counts
from lda_cgs import CollapseGibbsLda
import visualisation.pyLDAvis as pyLDAvis

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
multiplier = 1
n_topics = 20 * multiplier
n_docs = 100 * multiplier
vocab_size = 200 * multiplier
document_length = 50 * multiplier

alpha = 0.1
beta = 0.01    
n_samples = 200
n_burn = 0
n_thin = 1

random_state = RandomState(1234567890)
gen = LdaDataGenerator(alpha, make_plot=True)
df, vocab = gen.generate_from_file('../input/test1.csv', '../input/test1.words')

In [15]:
gibbs1 = CollapseGibbsLda(df, vocab, n_topics, alpha, beta, random_state=random_state, previous_model=None)
start_time = time.time()
gibbs1.run(n_burn, n_samples, n_thin, use_native=True)
print("--- TOTAL TIME %d seconds ---" % (time.time() - start_time))
print gibbs1.posterior_alpha

CGS LDA initialising
..........
Using Numba for LDA sampling
Preparing words
Preparing Z matrix
DONE
Sample 1   Log joint likelihood = -40361.524 
Sample 2   Log joint likelihood = -33684.924 
Sample 3   Log joint likelihood = -30332.865 
Sample 4   Log joint likelihood = -28611.731 
Sample 5   Log joint likelihood = -27428.915 
Sample 6   Log joint likelihood = -26771.378 
Sample 7   Log joint likelihood = -26202.570 
Sample 8   Log joint likelihood = -25637.589 
Sample 9   Log joint likelihood = -25284.896 
Sample 10   Log joint likelihood = -25072.341 
Sample 11   Log joint likelihood = -24769.868 
Sample 12   Log joint likelihood = -24480.904 
Sample 13   Log joint likelihood = -24274.089 
Sample 14   Log joint likelihood = -24078.263 
Sample 15   Log joint likelihood = -23911.653 
Sample 16   Log joint likelihood = -23786.088 
Sample 17   Log joint likelihood = -23619.600 
Sample 18   Log joint likelihood = -23579.539 
Sample 19   Log joint likelihood = -23490.796 
Sample 20   Log

In [23]:
gibbs1.print_topic_words()

Topic 0: gibbs1_word_2 (0.0841433021807), gibbs1_word_1 (0.0747975077882), gibbs1_word_0 (0.0623364485981), gibbs1_word_9 (0.0592211838006), gibbs1_word_127 (0.0561059190031), gibbs1_word_6 (0.0561059190031), gibbs1_word_7 (0.0561059190031), gibbs1_word_3 (0.0529906542056),
Topic 1: gibbs1_word_134 (0.121981707317), gibbs1_word_139 (0.112835365854), gibbs1_word_137 (0.109786585366), gibbs1_word_130 (0.100640243902), gibbs1_word_138 (0.100640243902), gibbs1_word_135 (0.0975914634146), gibbs1_word_131 (0.0945426829268), gibbs1_word_133 (0.0945426829268), gibbs1_word_136 (0.07625), gibbs1_word_132 (0.0732012195122),
Topic 2: gibbs1_word_143 (0.122590529248), gibbs1_word_140 (0.108662952646), gibbs1_word_149 (0.105877437326), gibbs1_word_144 (0.103091922006), gibbs1_word_146 (0.100306406685), gibbs1_word_142 (0.0919498607242), gibbs1_word_141 (0.0919498607242), gibbs1_word_147 (0.0919498607242), gibbs1_word_148 (0.0863788300836), gibbs1_word_145 (0.0808077994429),
Topic 3: gibbs1_word_164 

In [83]:
model = gibbs1
data = {}
data['topic_term_dists'] = model.topic_word_
data['doc_topic_dists'] = model.doc_topic_
data['doc_lengths'] = model.cd
data['vocab'] = model.vocab
data['term_frequency'] = np.sum(model.ckn, axis=0)    
vis_data = pyLDAvis.prepare(**data)  
pyLDAvis.show(vis_data) 

Using visualisation script defined in /LDAvis.js

Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [09/Jul/2015 17:58:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2015 17:58:15] "GET /LDAvis.css HTTP/1.1" 200 -



stopping Server...


127.0.0.1 - - [09/Jul/2015 17:58:15] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2015 17:58:15] "GET /LDAvis.js HTTP/1.1" 200 -
